In [1]:
!hostname

c0709a-s30.ufhpc


In [2]:
%load_ext autoreload
%autoreload 2

import os
os.environ['KMP_WARNINGS'] = 'off'
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import re

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

In [3]:
from scripts.nanohh4b.nanohh4b import Analysis
analysis = Analysis.load(
    '/home/ekoenig/analysis/studies/configs/nanohh4b/nanohh4b.yaml',
    signal=[],
    qcd=[],
    ttbar=[],
)

In [4]:
analysis.run(['init'])

0:00:00 [running ] init()


0it [00:00, ?it/s]


[WARNING] unable to open any files with filelist
           []


0it [00:00, ?it/s]


[WARNING] unable to open any files with filelist
           []


...

[WARNING] unable to open any files with filelist
           []



  0%|          | 0/1 [00:00<?, ?it/s]

Using local path: /cmsuf/data/store/user/ekoenig/root/eosuser.cern.ch/eos/user/m/mkolosov/Run2_HHTo4B_NTuples/UL2018/DeepJet_woTrgMatching_17Oct2023_2018_0L/data/jetht_tree.root


100%|██████████| 1/1 [00:00<00:00,  3.82it/s]


0:00:03 [  done  ] 0:00:03
        [disabled] set_bdisc_threshold(signal, bkg, data)
        [disabled] hh_mass_cut(signal, bkg, data)
        [disabled] load_feynnet(signal, bkg, data)
        [disabled] plot_jet_multiplicity(signal, bkg, data)
        [disabled] plot_reco_eff(signal)
        [disabled] plot_higgs(signal, bkg)
        [disabled] plot_region_vars(signal, bkg)
        [disabled] print_4btag_yields(signal, bkg, data)
        [disabled] plot_4b_control(signal, bkg, data)
        [disabled] print_3btag_yields(signal, bkg, data)
        [disabled] plot_3b_control(signal, bkg, data)
        [disabled] blind_data(data)
        [disabled] print_abcd_yields(signal, bkg, data)
        [disabled] plot_3btag_datamc(data, bkg)
        [disabled] train_bdt(data)
        [disabled] build_bkg_model(signal, bkg, data)
        [disabled] print_sr_yields(signal, bkg_model)
        [disabled] plot_limits(signal, bkg_model)
        [disabled] train_bdt_classifier(signal, bkg_model)
       

In [5]:
# analysis.data = analysis.data.subset(fraction=0.1, randomize=False)

In [6]:
analysis.run(['blind_data'])

0:00:00 [  done  ] init()
        [running ] set_bdisc_threshold(signal, bkg, data)
        [  done  ] 0:00:00
        [running ] hh_mass_cut(signal, bkg, data)
        [  done  ] 0:00:00
        [running ] load_feynnet(signal, bkg, data)
        [  done  ] 0:00:00
        [disabled] plot_jet_multiplicity(signal, bkg, data)
        [disabled] plot_reco_eff(signal)
        [disabled] plot_higgs(signal, bkg)
        [disabled] plot_region_vars(signal, bkg)
        [disabled] print_4btag_yields(signal, bkg, data)
        [disabled] plot_4b_control(signal, bkg, data)
        [disabled] print_3btag_yields(signal, bkg, data)
        [disabled] plot_3b_control(signal, bkg, data)
        [running ] blind_data(data)
0:00:04 [  done  ] 0:00:04
        [disabled] print_abcd_yields(signal, bkg, data)
        [disabled] plot_3btag_datamc(data, bkg)
        [disabled] train_bdt(data)
        [disabled] build_bkg_model(signal, bkg, data)
        [disabled] print_sr_yields(signal, bkg_model)
        [

In [7]:
tree = analysis.data[0]

In [8]:
tree.run

<Array [315366, 315366, ... 315713, 315713] type='3785883 * uint32'>

In [9]:
from scripts.nanohh4b.nanohh4b import h_dm

self = analysis

bdt = ABCD(
    features=['dHH_HH_regmass','dHH_HH_regpt'],
    a=lambda t : (h_dm(t) <  25) & (self.n_btag(t) == 4),
    b=lambda t : (h_dm(t) <  25) & (self.n_btag(t) == 3),
    c=lambda t : (h_dm(t) >= 25) & (h_dm(t) < 50) & (self.n_btag(t) == 4),
    d=lambda t : (h_dm(t) >= 25) & (h_dm(t) < 50) & (self.n_btag(t) == 3),
    **self.bdt_reweighter,
)